# Openstack Swift Object Store backend for zarr

This backend enables direct access to Swift Object Storage to read and write zarr files.

In [1]:
import os
import zarr
from zarrswift import SwiftStore

## zarr example

In [2]:
store = SwiftStore(container='demo', prefix='zarr-demo')
root = zarr.group(store=store, overwrite=True)
z = root.zeros('foo/bar', shape=(100, 100), chunks=(50, 50), dtype='i4')
z[:] = 42
z

<zarr.core.Array '/foo/bar' (100, 100) int32>

check array contents

In [3]:
z[:5, :5]

array([[42, 42, 42, 42, 42],
       [42, 42, 42, 42, 42],
       [42, 42, 42, 42, 42],
       [42, 42, 42, 42, 42],
       [42, 42, 42, 42, 42]], dtype=int32)

Listing store contents

In [4]:
store.listdir()

['.zgroup', 'foo']

In [5]:
store._walk()

['.zgroup',
 'foo/.zgroup',
 'foo/bar/.zarray',
 'foo/bar/0.0',
 'foo/bar/0.1',
 'foo/bar/1.0',
 'foo/bar/1.1']

Listing store contents using swift cli

In [6]:
! swift list demo

zarr-demo/.zgroup
zarr-demo/foo/.zgroup
zarr-demo/foo/bar/.zarray
zarr-demo/foo/bar/0.0
zarr-demo/foo/bar/0.1
zarr-demo/foo/bar/1.0
zarr-demo/foo/bar/1.1


## xarray example

In [7]:
import xarray as xr
import numpy as np

ds = xr.Dataset(
    {"foo": (('x', 'y'), np.random.rand(4, 5))},
    coords = {
        'x': [10, 20, 30, 40],
        'y': [1, 2, 3, 4, 5],
    },
)

ds

<xarray.Dataset>
Dimensions:  (x: 4, y: 5)
Coordinates:
  * x        (x) int64 10 20 30 40
  * y        (y) int64 1 2 3 4 5
Data variables:
    foo      (x, y) float64 0.2134 0.9704 0.8204 0.777 ... 0.5694 0.9138 0.7174

setup the store

In [8]:
store = SwiftStore(container='demo', prefix='xarray-demo')
store.listdir()

[]

saving dataset to store

In [9]:
ds.to_zarr(store=store, mode='w', consolidated=True)
store.listdir()

['.zattrs', '.zgroup', '.zmetadata', 'foo', 'x', 'y']

Loading dataset from store

In [10]:
ds2 = xr.open_zarr(store, consolidated=True)
ds2

<xarray.Dataset>
Dimensions:  (x: 4, y: 5)
Coordinates:
  * x        (x) int64 10 20 30 40
  * y        (y) int64 1 2 3 4 5
Data variables:
    foo      (x, y) float64 ...

## SwiftStore authentication

For authentication `SwiftStore` requires either (`authurl`, `user`, `key`) or pre-authenticated (`preauthurl`, `preauthtoken`) values.
If these values are not provided, it checks for corresponding environment variables (`ST_AUTH`, `ST_USER`, `ST_KEY`) or (`OS_STORAGE_URL`, `OS_AUTH_TOKEN`)

 
In the above examples, authentication is provided via environment variables

In [11]:
getenv = os.environ.get
print('authurl is None:', getenv("ST_AUTH") is None)
print('user is None:', getenv("ST_USER") is None)
print('key is None:', getenv("ST_KEY") is None)
print('preauthurl is None:', getenv("OS_STORAGE_URL") is None)
print('preauthtoken is None:', getenv("OS_AUTH_TOKEN") is None)

authurl is None: False
user is None: False
key is None: False
preauthurl is None: True
preauthtoken is None: True


After initial authentication, (`preauthurl`, `preauthtoken`) can be used to create new SwfitStore objects

In [12]:
preauthurl = store.conn.url
preauthtoken = store.conn.token

new_store = SwiftStore(container='demo', prefix='xarray-demo2', preauthurl=preauthurl, preauthtoken=preauthtoken)
ds.to_zarr(store=new_store, mode='w', consolidated=True)
new_store.listdir()

['.zattrs', '.zgroup', '.zmetadata', 'foo', 'x', 'y']

container listing using swift cli should contain 'xarray-demo2'

In [13]:
!swift list --delimiter '/' demo

xarray-demo/
xarray-demo2/
zarr-demo/


Instead of getting `preauthurl` and `preauthtoken` from a SwiftStore instance object, use the following function.

In [14]:
def acquire_token(authurl, user, key, token_life_in_seconds=24*60*60):
    'token_life_in_seconds determines for how long a token must be valid. currently set to 24 hours'
    import requests
    headers = {
        'X-Auth-User': user,
        'X-Auth-Key': key,
        'X-Auth-Lifetime': str(token_life_in_seconds),
    }
    r = requests.get(authurl, headers=headers)
    r.raise_for_status()
    preauthurl = r.headers['x-storage-url']
    preauthtoken = r.headers['x-auth-token']
    return preauthurl, preauthtoken

In [15]:
getenv = os.environ.get

storageurl, token = acquire_token(authurl=getenv('ST_AUTH'), user=getenv('ST_USER'), key=getenv('ST_KEY'))

print(storageurl is not None, token is not None)

True True


In [16]:
store = SwiftStore(container='demo', preauthurl=storageurl, preauthtoken=token)
store.listdir()

['xarray-demo', 'xarray-demo2', 'zarr-demo']

As this token is currently valid for a day (see acquire_token), use this instead of creating a new token for every new SwiftStore object.

For convenience, set the token life for a week and export them as environment variables in say '~/.bashrc'.